# Introduction

#### Latar Belakang
Dataset ini adalah heart failure clinical data. Data ini dikumpulkan dari 299 pasien dengan penyakit jantung gagal. Data ini digunakan untuk memprediksi apakah pasien akan meninggal dalam waktu 1 tahun setelah diagnosis.

#### Data yang Digunakan
Data ini bersumber dari Kaggle dengan link sebagai berikut : 
https://www.kaggle.com/datasets/andrewmvd/heart-failure-clinical-data

#### Objektif
Final Project 3 ini dibuat guna mengevaluasi konsep Ensemble sebagai berikut:
1. Mampu memahami konsep Classification dengan Ensemble Model
2. Mampu mempersiapkan data untuk digunakan dalam Ensemble Model
3. Mampu mengimplementasikan Ensemble Model untuk membuat prediksi

# Import Library

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn import preprocessing

import xgboost
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.svm import SVC 
from sklearn.tree import DecisionTreeClassifier 
from sklearn.ensemble import RandomForestClassifier 
from sklearn.naive_bayes import GaussianNB 
from scipy import stats

# Data Loading

In [ ]:
# Mengimpor dan membaca 5 data teratas pada dataset
df = pd.read_csv('heart_failure_clinical_records_dataset.csv')
df.head()

In [ ]:
# Menampilkan 5 data terbawah pada dataset
df.tail()

In [ ]:
# Menampilkan nama-nama kolom yang terdapat pada dataset ini
df.columns

In [ ]:
# Menampilkan informasi mengenai dataset
df.info()

In [ ]:
# Menampilkan jumlah kolom dan baris pada data
df.shape

# Data Cleaning

#### Missing Value

Mengecek missing value sebelum melakukan eksplorasi data lebih lanjut.

In [ ]:
df.isnull().sum()

Dataset ini bersih, tidak terdapat missing value didalamnya

In [ ]:
# Mengubah nama kolom DEATH_EVENT menjadi death_event
df = df.rename(columns={"DEATH_EVENT":"death_event"})

# Data Exploration

#### Central Tendency

In [ ]:
age = df["age"]
mean=age.mean()
median=age.median()
mode=age.mode()

# Print the result
print("Mean: ", mean)
print("Median: ", median)
print("Mode: ", mode)

Berdasarkan output di atas, dapat disimpulkan bahwa:
1. Mean usia pasien dengan penyakit jantung gagal adalah 60 tahun.
2. Median usia pasien dengan penyakit jantung gagal adalah 60 tahun.
3. Mode usia pasien dengan penyakit jantung gagal adalah 60 tahun.

#### Crosstab

In [ ]:
# Melihat proporsi data pada kolom Usia dan Kematian dengan menggunakan crosstab
pd.crosstab(df.death_event, df.age)

#### Grouping

In [ ]:
# Mengelompokkan Data Kategorikal dan Numerical
categorical_fit = ['anaemia','diabetes','high_blood_pressure', 'sex', 'smoking','DEATH_EVENT']
numerical_fit = ['age', 'creatinine_phosphokinase', 'ejection_fraction', 'platelets', 'serum_creatinine', 'serum_sodium', 'time']

#### Visualization

In [ ]:
# Split data menjadi dua kelompok berdasarkan death event
df_died = df[df["death_event"] == 1]
df_not_died = df[df["death_event"] == 0]

# Plot distribusi untuk setiap kelompok
plt.figure(figsize=(10, 6))
sns.histplot(x="age", hue="death_event", data=df, kde=True, palette="muted")
plt.title("Distribusi Usia pada death_event")
plt.legend(["Meninggal", "Tidak Meninggal"], loc="upper right")
plt.show()

Dari histogram tersebut, dapat disimpulkan bahwa : Kematian lebih sering terjadi pada usia tua. Hal ini terlihat pada puncak histogram yang berada pada rentang usia 60-70 tahun.


In [ ]:
# Plot distribusi gender pada pe
plt.figure(figsize=(10, 6))
sns.countplot(x="sex", data=df)
plt.title("Distribusi Jenis Kelamin pada death_event")
plt.show()

Visualisasi ini menunjukkan bahwa proporsi kematian pada pria lebih tinggi daripada wanita. Hal ini menunjukkan bahwa pria memiliki risiko kematian yang lebih tinggi.

#### Analisis Diabetes terhadap Resiko Kematian akibat Gagal Jantung

In [ ]:
Diabetes = df['diabetes'].value_counts(normalize=True)*100
Diabetes

Keterangan :
0 = Tidak mengidap diabetes
1 = Mengidap diabetes

In [ ]:
df1 = df[(df['diabetes'] == 0) & (df['death_event'] == 1)]
df2 = df[(df['diabetes'] == 1) & (df['death_event'] == 1)]

diabetes_death = pd.concat([df1, df2])

In [ ]:
fig, ax = plt.subplots(figsize=(15, 3))

diabetes_death['diabetes'].value_counts().plot.pie(autopct="%1.1f%%",
                                                   colors = ['thistle','orchid'],
                                                   startangle = 60,
                                                   ax=ax, 
                                                   shadow=True, 
                                                   explode=[0, 0.1])
ax.set_title('Persentase penderita diabetes yang meninggal')
plt.show()

Keterangan :

    0 = tidak mengidap diabetes, 
    1 = mengidap diabetes
    
Interpretasi:
- Terdapat 58,3% pasien yang tidak mengidap penyakit diabetes meninggal
- Terdapat 41,7% pasien yang mengidap penyakit diabetes meninggal

Pasien dengan riwayat diabetes memiliki resiko kematian yang lebih rendah dibanding pasien tanpa riwayat diabetes. Sehingga diabetes tidak berpengaruh terhadap resiko kematian gagal jantung

#### Analisis Perokok terhadap Resiko Kematian akibat Gagal Jantung

In [ ]:
Rokok = df['smoking'].value_counts(normalize=True)*100
Rokok

In [ ]:
df3 = df[(df['smoking'] == 0) & (df['death_event'] == 1)]
df4 = df[(df['smoking'] == 1) & (df['death_event'] == 1)]

smoking_death = pd.concat([df5, df6])

In [ ]:
fig, ax = plt.subplots(figsize=(15, 3))

smoking_death['smoking'].value_counts().plot.pie(autopct="%1.1f%%",
                                                   colors = ['thistle','orchid'],
                                                   startangle = 60,
                                                   ax=ax, 
                                                   shadow=True, 
                                                   explode=[0, 0.1])
ax.set_title('Persentase perokok yang meninggal')
plt.show()

Keterangan :

    0 = tidak merokok, 
    1 = merokok
    
Interpretasi:
- Terdapat 68,8% pasien yang tidak merokok meninggal
- Terdapat 31,2% pasien yang merokok meninggal

Pasien dengan riwayat perokok aktif memiliki resiko kematian yang lebih rendah dibanding pasien bukan perokok. Sehingga merokok tidak berpengaruh terhadap resiko kematian gagal jantung

#### Analisis Anemia terhadap Resiko Kematian akibat Gagal Jantung

In [ ]:
Anemia = df['anaemia'].value_counts(normalize=True)*100
Anemia

In [ ]:
df5 = df[(df['anaemia'] == 0) & (df['death_event'] == 1)]
df6 = df[(df['anaemia'] == 1) & (df['death_event'] == 1)]

anemia_death = pd.concat([df3, df4])

In [ ]:
fig, ax = plt.subplots(figsize=(15, 3))

anemia_death['anaemia'].value_counts().plot.pie(autopct="%1.1f%%",
                                                   colors = ['thistle','orchid'],
                                                   startangle = 60,
                                                   ax=ax, 
                                                   shadow=True, 
                                                   explode=[0, 0.1])
ax.set_title('Persentase penderita anemia yang meninggal')
plt.show()

Keterangan :

    0 = tidak mengidap anemia, 
    1 = mengidap anemia
    
Interpretasi:
- Terdapat 52,1% pasien yang tidak mengidap anemia meninggal
- Terdapat 47,9% pasien yang mengidap anemia meninggal

Pasien dengan riwayat anemia memiliki resiko kematian yang lebih rendah dibanding pasien tanpa riwayat anemia. Sehingga anemia tidak berpengaruh terhadap resiko kematian gagal jantung

#### Analisis Penderita Darah Tinggi terhadap Resiko Kematian akibat Gagal Jantung

In [ ]:
Darting = df['high_blood_pressure'].value_counts(normalize=True)*100
Darting

In [ ]:
df7 = df[(df['high_blood_pressure'] == 0) & (df['death_event'] == 1)]
df8 = df[(df['high_blood_pressure'] == 1) & (df['death_event'] == 1)]

Darting_death = pd.concat([df7, df8])

In [ ]:
fig, ax = plt.subplots(figsize=(15, 3))
Darting_death['high_blood_pressure'].value_counts().plot.pie(autopct="%1.1f%%",
                                                   colors = ['thistle','orchid'],
                                                   startangle = 60,
                                                   ax=ax, 
                                                   shadow=True, 
                                                   explode=[0, 0.1])
ax.set_title('Persentase penderita darah tinggi yang meninggal')
plt.show()

Keterangan :

    0 = tidak memiliki tekanan darah tinggi, 
    1 = memiliki tekanan darah tinggi
    
Interpretasi:
- Terdapat 59,4% pasien yang tidak menderita darah tinggi
- Terdapat 40,6% pasien yang menderita darah tinggi

Pasien dengan riwayat darah tinggi memiliki resiko kematian yang lebih rendah dibanding pasien tanpa riwayat darah tinggi. Sehingga Tekanan darah tinggi tidak berpengaruh terhadap resiko kematian gagal jantung

#### INSIGHT
Berdasarkan analisis diatas, dapat ditarik kesimpulan bahwa diabetes, anemia, merokok, dan tekanan darah tinggi tidak berpengaruh secara signifikan terhadap resiko kematian gagal jantung. 

#### Heatmap

In [ ]:
plt.figure(figsize=(15, 12))

sns.heatmap(df.corr(), annot=True, fmt=".2f", vmin=-1, vmax=1, square=True)

plt.title("Correlation Heatmap")

plt.xticks(rotation=45)
plt.yticks(rotation=0)
plt.show()

# Data Preprocessing

In [ ]:
X = df.drop("death_event",axis=1)
y = df["death_event"]

In [ ]:
# Membagi data menjadi train set dan test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Model Training

#### Logistic Regression

In [ ]:
lr = LogisticRegression()
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)

#### K-Nearest Neighbors

In [ ]:
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
y_pred_knn = knn.predict(X_test)

#### Support Vector Machine (SVM)

In [ ]:
svm = SVC()
svm.fit(X_train, y_train)
y_pred_svm = svm.predict(X_test)

#### Naive Bayes

In [ ]:
nb = GaussianNB()
nb.fit(X_train, y_train)
y_pred_nb = nb.predict(X_test)

#### Decision Tree

In [ ]:
dtc = DecisionTreeClassifier()
dtc.fit(X_train, y_train)
y_pred_dtc = dtc.predict(X_test)

#### Random Forest Classifier

In [ ]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

#### Gradient Boosting Classifier

In [ ]:
gb = GradientBoostingClassifier()
gb.fit(X_train, y_train)
y_pred_gb = gb.predict(X_test)

#### Extra Trees Classifier

In [ ]:
et = ExtraTreesClassifier()
et.fit(X_train, y_train)
y_pred_et = et.predict(X_test)

# Model Evaluation

#### Logistic Regression

In [ ]:
# Evaluasi model Logistic Regression
accuracy_lr = accuracy_score(y_test, y_pred_lr)
print("Accuracy (Logistic Regression):", accuracy_lr)

In [ ]:
# Menampilkan classification report dan confusion matrix
print("\nClassification Report (Logistic Regression):")
print(classification_report(y_test, y_pred_lr))

print("\nConfusion Matrix (Logistic Regression):")
print(confusion_matrix(y_test, y_pred_lr))

#### K-Nearest Neighbour

In [ ]:
# Evaluasi model KNN
accuracy_knn = accuracy_score(y_test, y_pred_knn)
print("Accuracy (Logistic Regression):", accuracy_knn)

In [ ]:
# Menampilkan classification report dan confusion matrix
print("\nClassification Report (KNN):")
print(classification_report(y_test, y_pred_knn))

print("\nConfusion Matrix (KNN):")
print(confusion_matrix(y_test, y_pred_knn))

#### Support Vector Machine (SVM)

In [ ]:
# Evaluasi model SVM
accuracy_svm = accuracy_score(y_test, y_pred_svm)
print("Accuracy (Logistic Regression):", accuracy_svm)

In [ ]:
# Menampilkan classification report dan confusion matrix
print("\nClassification Report (SVM):")
print(classification_report(y_test, y_pred_svm))

print("\nConfusion Matrix (SVM):")
print(confusion_matrix(y_test, y_pred_svm))

#### Naive Bayes

In [ ]:
# Evaluasi model Naive Bayes
accuracy_nb = accuracy_score(y_test, y_pred_nb)
print("Accuracy (Logistic Regression):", accuracy_nb)

In [ ]:
# Menampilkan classification report dan confusion matrix
print("\nClassification Report (Naive Bayes):")
print(classification_report(y_test, y_pred_nb))

print("\nConfusion Matrix (Naive Bayes):")
print(confusion_matrix(y_test, y_pred_nb))

#### Decision Tree

In [ ]:
# Evaluasi model Decision Tree
accuracy_dt = accuracy_score(y_test, y_pred_dt)
print("Accuracy (Logistic Regression):", accuracy_dt)

In [ ]:
# Menampilkan classification report dan confusion matrix
print("\nClassification Report (Decision Tree):")
print(classification_report(y_test, y_pred_dt))

print("\nConfusion Matrix (Naive Bayes):")
print(confusion_matrix(y_test, y_pred_dt))

#### Random Forest Classifier

In [ ]:
# Evaluasi model Random Forest Classifier
accuracy_rf = accuracy_score(y_test, y_pred_rf)
print("Accuracy (Logistic Regression):", accuracy_rf)

In [ ]:
# Menampilkan classification report dan confusion matrix
print("\nClassification Report (Random Forest):")
print(classification_report(y_test, y_pred_rf))

print("\nConfusion Matrix (Random Forest):")
print(confusion_matrix(y_test, y_pred_rf))

#### Gradient Boosting Classifier

In [ ]:
# Evaluasi model Gradient Boosting Classifier
accuracy_gb = accuracy_score(y_test, y_pred_gb)
print("Accuracy (Logistic Regression):", accuracy_gb)

In [ ]:
# Menampilkan classification report dan confusion matrix
print("\nClassification Report (Gradient Boosting):")
print(classification_report(y_test, y_pred_gb))

print("\nConfusion Matrix (Gradient Boosting):")
print(confusion_matrix(y_test, y_pred_gb))

#### Extra Tree Classifier

In [ ]:
# Evaluasi model Extraa Tree Classifier
accuracy_et = accuracy_score(y_test, y_pred_et)
print("Accuracy (Logistic Regression):", accuracy_et)

In [ ]:
# Menampilkan classification report dan confusion matrix
print("\nClassification Report (Extraa Tree):")
print(classification_report(y_test, y_pred_et))

print("\nConfusion Matrix (Extra Tree):")
print(confusion_matrix(y_test, y_pred_et))

# Kesimpulan

Berdasarkan data yang telah dianalisis, dapat disimpulkan bahwa:
1. Secara umum, kematian lebih umum terjadi pada usia tua. Hal ini terlihat dari puncak histogram yang berada pada rentang usia 70-80 tahun.
2. Perbedaan distribusi usia pada peristiwa kematian antara kelompok yang meninggal dan yang tidak meninggal dapat dikaitkan dengan faktor-faktor seperti penyakit kronis dan usia.

Berdasarkan visualisasi distribusi jenis kelamin pada peristiwa kematian, dapat disimpulkan bahwa:

Proporsi kematian pada pria lebih tinggi daripada proporsi kematian pada wanita. Hal ini menunjukkan bahwa pria memiliki risiko kematian yang lebih tinggi akibat gagal jantung.
